In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
geonames_fr = pd.read_csv("./FR/FR.txt", sep='\t', lineterminator='\r', low_memory=False, encoding='UTF-8')

KeyboardInterrupt: 

In [2]:
# read geonames DE data
geonames_de = pd.read_csv('./DE/DE_mod.csv', delimiter='\t', low_memory=False, )
geonames_de=geonames_de.drop(['cc2', 'dem', 'timezone', 'elevation'], axis=1)

geonames_de['parent']=0
geonames_de['adm4_feature']=0
geonames_de['adm3_feature']=0
geonames_de['adm2_feature']=0
geonames_de['adm1_feature']=0


# read geonames hierarchy data
#hierarchy = pd.read_csv('./hierarchy.csv', delimiter='\t', names=["parentId", "childId", "type"])
#hierarchy = hierarchy.drop("type", axis=1)

print(geonames_de.columns)
#print(hierarchy.columns)

Index(['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude',
       'longitude', 'feature_class', 'feature_code', 'country_code',
       'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code',
       'population', 'modification_date', 'parent', 'adm4_feature',
       'adm3_feature', 'adm2_feature', 'adm1_feature'],
      dtype='object')


In [3]:
geonames_de

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population,modification_date,parent,adm4_feature,adm3_feature,adm2_feature,adm1_feature
0,1529666,Bahnhof Grenzau,Bahnhof Grenzau,"Bahnhof Grenzau,Grenzau",50.45663,7.66505,S,RSTN,DE,8.0,0.0,7143.0,7143032.0,0,2020-10-14,0,0,0,0,0
1,2038682,Bahnhof Annaburg,Bahnhof Annaburg,"Annaburg,Bahnhof Annaburg,Bahnhof Annaburg West",51.72858,13.03311,S,RSTN,DE,11.0,NaN,NaN,NaN,0,2020-10-14,0,0,0,0,0
2,2657946,Wyhlen,Wyhlen,Wyhlen,47.54729,7.69331,P,PPLX,DE,1.0,83.0,8336.0,8336105.0,0,2020-11-12,0,0,0,0,0
3,2658739,Schiener Bach,Schiener Bach,Schiener Bach,47.6802,8.86131,H,STM,DE,0.0,NaN,NaN,NaN,0,2015-09-06,0,0,0,0,0
4,2659829,Lunkenbach,Lunkenbach,"Lunckenbach,Lunkenbach",47.68136,8.84938,H,STM,DE,0.0,NaN,NaN,NaN,0,2015-09-06,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198186,12216996,Reuterstrasse Tower,Reuterstrasse Tower,"Krepostnaja bashnja na Rejtershtrasse,Menara b...",49.76377,9.94506,S,TOWR,DE,2.0,96.0,9663.0,9663000.0,0,2021-02-21,0,0,0,0,0
198187,12217000,Seilerstrasse Defensive Wall,Seilerstrasse Defensive Wall,"Cinta muraria su Seilerstrasse,Gorodskaja sten...",49.76472,9.95306,S,WALL,DE,2.0,96.0,9663.0,9663000.0,0,2021-02-21,0,0,0,0,0
198188,12217003,Salmann's Tower,Salmann's Tower,"Berbenteng Menara Salmann,Krepostnaja bashnja ...",49.76187,9.95225,S,TOWR,DE,2.0,96.0,9663.0,9663000.0,0,2021-02-21,0,0,0,0,0
198189,12217011,Werkingstrasse Defensive Wall,Werkingstrasse Defensive Wall,"Cinta muraria su Werkingstrasse,Gorodskaja ste...",49.76375,9.95367,S,WALL,DE,2.0,96.0,9663.0,9663000.0,0,2021-02-22,0,0,0,0,0


In [4]:
%%time

regexp = re.compile(r'\(ADM\)+\\d\\b')
for row in geonames_de[0:].itertuples(index=True, name="Feature"):
    parentIsSet=False
    dictRow = row._asdict()
    #print(dictRow['geonameid'])
    #print(dictRow['name'])
    if(pd.isna(dictRow['feature_code']) == False and regexp.search(dictRow['feature_code'])==None):
        # search for parent feature by ascending the feature hierarchy
        adm4_parent = geonames_de[(geonames_de['feature_code']=='ADM4') & (geonames_de['admin1_code']==dictRow['admin1_code']) & (geonames_de['admin2_code']==dictRow['admin2_code']) & (geonames_de['admin3_code']==dictRow['admin3_code']) & (geonames_de['admin4_code']==dictRow['admin4_code'])]
        if(len(adm4_parent)==1):
            geonames_de.at[row[0],'adm4_feature'] = adm4_parent['geonameid']
            geonames_de.at[row[0],'parent'] = adm4_parent['geonameid']
            parentIsSet=True

        adm3_parent = geonames_de[(geonames_de['feature_code']=='ADM3') & (geonames_de['admin1_code']==dictRow['admin1_code']) & (geonames_de['admin2_code']==dictRow['admin2_code']) & (geonames_de['admin3_code']==dictRow['admin3_code'])]
        if(len(adm3_parent)==1):
            geonames_de.at[row[0],'adm3_feature'] = adm3_parent['geonameid']
            if(parentIsSet==False):
                geonames_de.at[row[0],'parent'] = adm3_parent['geonameid']
                parentIsSet=True

        adm2_parent = geonames_de[(geonames_de['feature_code']=='ADM2') & (geonames_de['admin1_code']==dictRow['admin1_code']) & (geonames_de['admin2_code']==dictRow['admin2_code'])]
        if(len(adm2_parent)==1):
            geonames_de.at[row[0],'adm2_feature'] = adm2_parent['geonameid']
            if(parentIsSet==False):
                geonames_de.at[row[0],'parent'] = adm2_parent['geonameid']
                parentIsSet=True

        adm1_parent = geonames_de[(geonames_de['feature_code']=='ADM1') & (geonames_de['admin1_code']==dictRow['admin1_code'])]
        if(len(adm1_parent)==1):
            geonames_de.at[row[0],'adm1_feature'] = adm1_parent['geonameid']
            if(parentIsSet==False):
                geonames_de.at[row[0],'parent'] = adm1_parent['geonameid']
                parentIsSet=True

        # check if there is no hierarchy at all -> then assign the country ('DE') as parent
        if(len(adm4_parent['geonameid'])==0 and len(adm3_parent['geonameid'])==0 and len(adm2_parent['geonameid'])==0 and len(adm1_parent['geonameid'])==0):
            geonames_de.at[row[0],'parent'] = 2921044

CPU times: user 2h 22min 18s, sys: 13.3 s, total: 2h 22min 31s
Wall time: 2h 23min 35s


In [5]:
geonames_de[0:25]

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population,modification_date,parent,adm4_feature,adm3_feature,adm2_feature,adm1_feature
0,1529666,Bahnhof Grenzau,Bahnhof Grenzau,"Bahnhof Grenzau,Grenzau",50.45663,7.66505,S,RSTN,DE,8.0,0.0,7143.0,7143032.0,0,2020-10-14,6557890,6557890,3247457,0,2847618
1,2038682,Bahnhof Annaburg,Bahnhof Annaburg,"Annaburg,Bahnhof Annaburg,Bahnhof Annaburg West",51.72858,13.03311,S,RSTN,DE,11.0,NaN,NaN,NaN,0,2020-10-14,2945356,0,0,0,2945356
2,2657946,Wyhlen,Wyhlen,Wyhlen,47.54729,7.69331,P,PPLX,DE,1.0,83.0,8336.0,8336105.0,0,2020-11-12,6555903,6555903,3214108,2925180,2953481
3,2658739,Schiener Bach,Schiener Bach,Schiener Bach,47.6802,8.86131,H,STM,DE,0.0,NaN,NaN,NaN,0,2015-09-06,2921044,0,0,0,0
4,2659829,Lunkenbach,Lunkenbach,"Lunckenbach,Lunkenbach",47.68136,8.84938,H,STM,DE,0.0,NaN,NaN,NaN,0,2015-09-06,2921044,0,0,0,0
5,2744273,Witte Venn,Witte Venn,"Witte Veen,Witte Venn",52.15,6.88333,H,MRSH,DE,0.0,NaN,NaN,NaN,0,2014-08-05,2921044,0,0,0,0
6,2744666,Westerwoldsche A,Westerwoldsche A,"Westerwoldsche A,Westerwoldsche Aa,Westerwolds...",53.23333,7.2,H,STMC,DE,0.0,NaN,NaN,NaN,0,2014-08-05,2921044,0,0,0,0
7,2745605,Hoge Veenkanal,Hoge Veenkanal,"Hoge Veenkanal,Verlangde Hoogeveensche Vaart,V...",52.73333,6.51667,H,CNL,DE,0.0,NaN,NaN,NaN,0,2014-08-05,2921044,0,0,0,0
8,2746549,Schwalm,Schwalm,"Schwalm,Swalm,Zwalm,Zwalmse Beek",51.25,6.01667,H,STM,DE,0.0,NaN,NaN,NaN,0,2014-08-05,2921044,0,0,0,0
9,2747513,Grenzaa,Grenzaa,"Aa,De Aa Suderbeeke,Grenzaa,Schoonebeekerdiep,...",52.65,6.76667,H,STM,DE,0.0,NaN,NaN,NaN,0,2020-07-31,2921044,0,0,0,0


In [6]:
geonames_de.to_csv("DE_mod_final.csv")